In [ ]:
import glob
import pickle

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib inline

# Transform

In [ ]:
def draw_lines(img, points, color=[255, 0, 0], thickness=2):
    pt1 = points[0]
    pt2 = points[1]
    pt3 = points[2]
    pt4 = points[3]

    cv2.line(img, (pt1[0], pt1[1]), (pt2[0], pt2[1]), color, thickness)
    cv2.line(img, (pt2[0], pt2[1]), (pt3[0], pt3[1]), color, thickness)
    cv2.line(img, (pt3[0], pt3[1]), (pt4[0], pt4[1]), color, thickness)
    cv2.line(img, (pt4[0], pt4[1]), (pt1[0], pt1[1]), color, thickness)

In [ ]:
# x2-x1/y2-y1 = x2-x3/y2-y3
def get_x3(x1, x2, y1, y2, y3):
    x3 = x2-(((x2-x1)/(y2-y1))*(y2-y3))
    print(x3)
    return x3

In [ ]:
images_directory = 'test_images/'
image_names_prefix = 'straight_lines'
image_file_extension = '.jpg'
images = glob.glob(images_directory + image_names_prefix + '*' + image_file_extension)

transform_pickle_path = 'wide_transform_pickle.p'

dist_pickle_path = 'wide_dist_pickle.p'
with open(dist_pickle_path, mode='rb') as f:
    dist_pickle = pickle.load(f)
    
mtx, dist = dist_pickle['mtx'], dist_pickle['dist']

y1 = 700 # 720, 650
y2 = 465
y3 = 460

left_x1 = 220 # 200, 300
left_x2 = 573 
left_x3 = get_x3(left_x1, left_x2, y1, y2, y3)

right_x1 = 1060 # 1100, 1000
right_x2 = 710
right_x3 = get_x3(right_x1, right_x2, y1, y2, y3)

src_pt1 = [left_x3, y3]
src_pt2 = [right_x3, y3]
src_pt3 = [right_x1, y1]
src_pt4 = [left_x1, y1]
src = np.float32([src_pt1, src_pt2, src_pt3, src_pt4])

dst_pt1 = [300, 0]
dst_pt2 = [1000, 0]
dst_pt3 = [1000, 720]
dst_pt4 = [300, 720]
dst = np.float32([dst_pt1, dst_pt2, dst_pt3, dst_pt4])

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')
    
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(img, src)
    sub2.imshow(img)
    if idx == 0:
        sub2.set_title('Original Images with source points')

fig.tight_layout()
plt.show()

In [ ]:
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    img_size = img.shape[1::-1]
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    draw_lines(img, src)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images with source points')

    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(warped, dst)
    sub2.imshow(warped)
    if idx == 0:
        sub2.set_title('Transformed Images with destination points')

fig.tight_layout()
plt.show()

In [ ]:
images = glob.glob('test_images/test*.jpg')

fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img, cmap='gray')
    sub1.set_xticks(())
    sub1.set_yticks(())

    # Draw and display the undistorted images
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(warped, cmap='gray')
    sub2.set_xticks(())
    sub2.set_yticks(())

fig.tight_layout()
plt.show()

# Threshold

In [ ]:
# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(175, 250), sx_thresh=(20, 100), gray=True):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    if gray == True:
        binary_output = np.zeros_like(sxbinary)
        binary_output[(s_binary == 1) | (sxbinary == 1)] = 1
        return binary_output
    else:
        # Stack each channel
        # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
        # be beneficial to replace this channel with something else.
        color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
        return color_binary

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]

    # Draw and display the original images
    wraped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(wraped)
    
    # Apply each of the thresholding functions  
    wraped_binary = pipeline(img, gray=False)
    wraped_binary = cv2.warpPerspective(wraped_binary, M, img_size, flags=cv2.INTER_LINEAR)
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(wraped_binary, cmap='gray')

fig.tight_layout()
plt.show()